# User behavior as a predictor for input accuracy
Johann Miller, University of Maryland Makeability Lab

### Introduction
[Project Sidewalk](http://sidewalk.umiacs.umd.edu) is an online platform for identifying accessibility problems in sidewalks. Users navigate streets using Google Street View, and place labels on issues such as crosswalks without curbramps, uneven pavement, and obstacles blocking the path. In order to ensure some level of accuracy in the data, Project Sidewalk can use a couple tools. Each user has to complete a tutorial before they can begin to report problems. Another option is ground truth seeding, where users place labels in a region that already has established answers. If the user enters data that doesn't match the ground truth, then all of the data they entered can be flagged.

Here, we will investigate another possible option: using the user's interactions with the tool to predict their accuracy. Consider a user who is inactive for long periods, and barely uses any of the tool's features. This user probably gives worse input than a user who works consistently and employs all of the tool's features. Aspects of interaction include mouse movement, keypresses, and others that we can collect while users place labels. If accurate labels correspond to a certain type of usage, then these features could predict the accuracy of a user even in non ground truth regions.

To see if this is possible, we'll use data from ground truth regions to train and test a classifier.

### Setup python notebook

In [1]:
import pandas as pd
import psycopg2 as pg

# connect to volunteer database
vol_con = pg.connect(database="sidewalk", user="sidewalk", password="sidewalk", host="localhost", port="5432" )

# connect to turker database
turk_con = pg.connect(database="sidewalk_turker", user="sidewalk", password="sidewalk", host="localhost", port="5432" )

In [16]:
%%html
<style>
img {display: inline-block; float='left'}
</style>

### Features
Project Sidewalk has logs for variety of user events. The events range from low-level (mouse movements, clicks, etc.) to high-level (zoom in/out, changing label mode, etc.). If we query the interaction table, we can see all the event types.

In [3]:
event_types = pd.read_sql(
'''
SELECT DISTINCT action
FROM audit_task_interaction
''', vol_con)

print("Number of event types:", len(event_types))
event_types.head()

Number of event types: 109


,action
0,ModeSwitch_Walk
1,KeyboardShortcut_ModeSwitch_CurbRamp
2,ViewControl_DoubleClick
3,KeyboardShortcut_ClickOk
4,KeyUp


For each user session, we have a collection of the events that were triggered. In order to compare two sessions, we can look at the total number of each type of event. We also look at the mean and standard deviation of the number of events per Google Street View panorama. This way, a user session in a large region can be compared fairly to a session in a smaller region since the former will have more panoramas.

We can load in these event counts from `features.csv`. This file was created by **TODO**.

In [4]:
features = pd.read_csv('features.csv', index_col=0)
features.sample(n=5)

,Click_LabelDelete_per_pan_mean,Click_LabelDelete_per_pan_std,Click_LabelDelete_total,Click_ModeSwitch_CurbRamp_per_pan_mean,Click_ModeSwitch_CurbRamp_per_pan_std,Click_ModeSwitch_CurbRamp_total,Click_ModeSwitch_NoCurbRamp_per_pan_mean,Click_ModeSwitch_NoCurbRamp_per_pan_std,Click_ModeSwitch_NoCurbRamp_total,Click_ModeSwitch_NoSidewalk_per_pan_mean,...,ViewControl_DoubleClick_total,ViewControl_MouseDown_per_pan_mean,ViewControl_MouseDown_per_pan_std,ViewControl_MouseDown_total,ViewControl_MouseUp_per_pan_mean,ViewControl_MouseUp_per_pan_std,ViewControl_MouseUp_total,WalkTowards_per_pan_mean,WalkTowards_per_pan_std,WalkTowards_total
322,0.000000,0.00000,0,6.000000,1.414214,12,1.000000,0.000000,1,0,...,1,5.812500,8.788013,93,5.500000,8.082904,88,0,0,0
207,0.000000,0.00000,0,0.000000,0.000000,0,0.000000,0.000000,0,0,...,0,0.000000,0.000000,0,0.000000,0.000000,0,0,0,0
67,0.000000,0.00000,0,0.000000,0.000000,0,0.000000,0.000000,0,0,...,0,0.000000,0.000000,0,0.000000,0.000000,0,0,0,0
92,1.000000,0.00000,5,3.555556,1.998207,224,1.454545,0.687552,16,0,...,2,2.369565,3.039742,1090,2.336957,2.983167,1075,0,0,0
23,1.333333,0.57735,4,1.874172,1.318103,283,1.323529,0.534880,45,0,...,4,2.286598,2.967163,1109,2.255670,2.831456,1094,0,0,0


### Labels
We need to rate each user session on its accuracy compared to the ground truth. We do so by counting the number of true positives, false positives, true negatives, and false negatives from the session. These are defined as follows:

#### True positive
The user correctly places a label. Here, the green icon is a label for a curb ramp. The user placed it correctly, so this is a true positive.

![true positive](images/true-pos.png)

#### False positive
The user places a label incorrectly. Here, the user placed a green icon to identify a curb ramp, but none are present.

![false positive](images/false-pos.png)

#### True negative
There was nothing to label, and the user didn't label anything. This can be calculated with varying granularity: 

![true negative](images/true-neg.png)

#### False negative
There was something to label, but the user missed it. Here, there is a curb ramp with no label.

![false negative](images/false-neg.png)